In [ ]:
import ev3dev2.motor
from ev3dev2.motor import LargeMotor, OUTPUT_A, OUTPUT_B, OUTPUT_C, SpeedPercent
from ev3dev2.sound import Sound



class GarbageSortController:
    # motor controller for sorting garbage
   def __init__(self):
       try:
           # adjust output ports for motors
           self.motor_recyclable = LargeMotor(OUTPUT_A)
           self.motor_compost = LargeMotor(OUTPUT_B)
           self.motor_trash = LargeMotor(OUTPUT_C)
           self.sound = Sound()
           self.motors_available = True
           print("EV3 motors initialized successfully")
       except Exception as e:
           print(f"EV3 motors not available: {e}")
           self.motors_available = False

       # motor settings
       self.default_speed = SpeedPercent(50)
       self.default_degrees = 90

   def handle_Battery(self):
       """Handle battery classification"""
       if self.motors_available:
           self.motor_recyclable.on_for_degrees(self.default_speed, self.default_degrees)
       else:
           print("Motor action: Battery -> Recyclable bin")
       # self.sound.speak("Battery")

   def handle_Glass(self):
       """Handle glass classification"""
       if self.motors_available:
           self.motor_recyclable.on_for_degrees(self.default_speed, self.default_degrees)
       else:
           print("Motor action: Glass -> Recyclable bin")
       # self.sound.speak("Glass")


   def handle_Metal(self):
       """Handle trash classification"""
       self.motor_trash.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Trash")

   def handle_Organic_Waste(self):
       """Handle organic waste classification"""
       self.motor_compost.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Organic Waste")


   def handle_Paper(self):
       """Handle paper classification"""
       self.motor_recyclable.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Paper")

   def handle_Plastic(self):
       """Handle plastic classification"""
       self.motor_recyclable.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Plastic")


   def handle_Textiles(self):
       """Handle textiles classification"""
       self.motor_recyclable.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Textiles")

   def handle_Trash(self):
       """Handle trash classification"""
       self.motor_trash.on_for_degrees(self.default_speed, self.default_degrees)
       # self.sound.speak("Trash")


   def handle_classification(self, label):
       """Main handler that routes classifications to appropriate motors"""
       print(f"Handling classification: {label}")  # debug output

       if label == "battery":
           self.handle_Battery()
       elif label == "glass":
           self.handle_Glass()
       elif label == "metal":
           self.handle_Metal()
       elif label == "organic_waste":
           self.handle_Organic_Waste()
       elif label == "paper_cardboard":
           self.handle_Paper()
       elif label == "plastic":
           self.handle_Plastic()
       elif label == "textiles":
           self.handle_Textiles()
       elif label == "trash":
           self.handle_Trash()
       else:
           print(f"Unknown classification: {label}")  # debug for unknown labels
   def set_motor_settings(self, speed_percent=50, degrees=90):
       self.default_speed = SpeedPercent(speed_percent)
       self.default_degrees = degrees

   def stop_all_motors(self):
       if self.motors_available:
           self.motor_recyclable.stop()
           self.motor_compost.stop()
           self.motor_trash.stop()
       else:
           print("Motor action: Stop all motors")


# global instance for easy import
garbage_sort_controller = GarbageSortController()

In [ ]:
import time
from ev3dev2.motor import LargeMotor, OUTPUT_A, OUTPUT_B, SpeedPercent
from ev3dev2.sound import Sound

class GarbageSortController:
    """Controls EV3 motors for trap door sorting: compost center, trash right, recycle left."""

    def __init__(self):
        try:
            # Motor A controls flaps left/right; Motor B controls trap door up/down
            self.flap_motor = LargeMotor(OUTPUT_A)
            self.trapdoor_motor = LargeMotor(OUTPUT_B)
            self.sound = Sound()
            self.motors_available = True
            print("EV3 motors initialized successfully.")
        except Exception as e:
            print(f"EV3 motors not available: {e}")
            self.motors_available = False

        # Default operational settings
        self.speed = SpeedPercent(50)
        self.flap_deg = 90   # adjust to move item fully left/right
        self.trap_deg = 180  # adjust to fully open/close trap door

    def open_trap(self):
        if self.motors_available:
            self.trapdoor_motor.on_for_degrees(self.speed, self.trap_deg)
            time.sleep(0.3)
        else:
            print(">>> Trap door opening...")

    def close_trap(self):
        if self.motors_available:
            self.trapdoor_motor.on_for_degrees(self.speed, -self.trap_deg)
            time.sleep(0.3)
        else:
            print(">>> Trap door closing...")

    def route_left(self):
        """Rotate flap to left (Recycle), then return to neutral."""
        if self.motors_available:
            self.flap_motor.on_for_degrees(self.speed, -self.flap_deg)
            time.sleep(0.2)
            self.flap_motor.on_for_degrees(self.speed, self.flap_deg)
        else:
            print(">>> Routing to LEFT (Recycle)")

    def route_right(self):
        """Rotate flap to right (Trash), then return to neutral."""
        if self.motors_available:
            self.flap_motor.on_for_degrees(self.speed, self.flap_deg)
            time.sleep(0.2)
            self.flap_motor.on_for_degrees(self.speed, -self.flap_deg)
        else:
            print(">>> Routing to RIGHT (Trash)")

    def route_middle(self):
        """Compost: no flap movement—just open trap door and let item drop."""
        print(">>> Routing to CENTER compost.")

    def handle_classification(self, label):
        """Call this with the predicted label from your model."""
        print(f"[INFO] Received classification: '{label}'")

        # If label is restricted, keep trap closed
        if label in ("battery", "glass", "textiles"):
            print(f"[INFO] '{label}' is restricted—no movement.")
            return

        # Open trap door for allowed items
        self.open_trap()

        if label == "compost":
            self.route_middle()
        elif label == "trash":
            self.route_right()
        elif label in ("paper_cardboard", "plastic", "metal"):
            self.route_left()
        else:
            print(f"[WARNING] Unknown label '{label}'. Closing trap without routing.")

        # Close the trap door
        self.close_trap()

    def set_motor_settings(self, speed_percent=50, degrees=90):
        """Optionally call this to adjust flap rotation and motor speed."""
        self.speed = SpeedPercent(speed_percent)
        self.flap_deg = degrees
        print(f"[CONFIG] Updated speed to {speed_percent}%, flap degrees to {degrees}")

    def stop_all_motors(self):
        """Emergency stop: halt any motor movement."""
        if self.motors_available:
            self.flap_motor.stop()
            self.trapdoor_motor.stop()
        print("[STOP] All motors stopped.")

# Global instance for easy import in your main script:
garbage_sort_controller = GarbageSortController()


In [ ]:
import time
from ev3dev2.motor import LargeMotor, OUTPUT_A, OUTPUT_B, OUTPUT_C, OUTPUT_D, SpeedPercent

class GarbageSortController:
    """Sequence: panels → rods → trap opens → trash drops → trap closes → rods retract → panels reset."""

    def __init__(self):
        try:
            self.panel_left = LargeMotor(OUTPUT_A)
            self.panel_right = LargeMotor(OUTPUT_B)
            self.rod_motor = LargeMotor(OUTPUT_C)
            self.trap_motor = LargeMotor(OUTPUT_D)
            self.motors_available = True
            print("Motors initialized.")
        except Exception as e:
            print(f"Motor init failed: {e}")
            self.motors_available = False

        self.speed = SpeedPercent(50)
        self.panel_deg = 45
        self.rod_deg = 40
        self.trap_deg = 95

    def shift_panels(self, direction):
        """45° shift: 'left', 'right', or None."""
        if not self.motors_available:
            print(f"Panels move {direction or 'none'} (simulated).")
            return
        deg = self.panel_deg if direction == 'left' else -self.panel_deg if direction == 'right' else 0
        if deg:
            self.panel_left.on_for_degrees(self.speed, deg, block=False)
            self.panel_right.on_for_degrees(self.speed, deg)
        time.sleep(0.2)

    def reset_panels(self):
        """Return panels to center."""
        if not self.motors_available:
            print("Panels reset to center.")
            return
        self.panel_left.on_for_degrees(self.speed, -self.panel_deg, block=False)
        self.panel_right.on_for_degrees(self.speed, -self.panel_deg)
        time.sleep(0.2)

    def extend_rods(self):
        """Raise support rods."""
        if not self.motors_available:
            print("Support rods extending.")
            return
        self.rod_motor.on_for_degrees(self.speed, self.rod_deg)
        time.sleep(0.2)

    def retract_rods(self):
        """Lower rods."""
        if not self.motors_available:
            print("Support rods retracting.")
            return
        self.rod_motor.on_for_degrees(self.speed, -self.rod_deg)
        time.sleep(0.2)

    def open_trap(self):
        """Open trap by moving downward."""
        if not self.motors_available:
            print("Trap doors opening.")
            return
        self.trap_motor.on_for_degrees(self.speed, -self.trap_deg)
        time.sleep(0.2)

    def close_trap(self):
        """Close trap by moving upward."""
        if not self.motors_available:
            print("Trap doors closing.")
            return
        self.trap_motor.on_for_degrees(self.speed, self.trap_deg)
        time.sleep(0.2)

    def handle_classification(self, label):
        print(f"→ Classified: {label}")
        if label in ("glass", "textiles", "battery"):
            print("Item not allowed.")
            return

        direction = {
            "trash": "right",
            "compost": None,
            "paper_cardboard": "left",
            "plastic": "left",
            "metal": "left"
        }.get(label)

        if direction is None and label not in ("compost", "trash"):
            print(f"Unknown label '{label}'; aborting.")
            return

        self.shift_panels(direction)
        self.extend_rods()
        self.open_trap()
        time.sleep(0.5)  # allow trash to fall
        self.close_trap()
        self.retract_rods()
        self.reset_panels()

    def set_motor_settings(self, speed=50, panel_deg=45, rod_deg=90, trap_deg=80):
        self.speed = SpeedPercent(speed)
        self.panel_deg = panel_deg
        self.rod_deg = rod_deg
        self.trap_deg = trap_deg
        print(f"[CONFIG] speed {speed}%, panel {panel_deg}°, rod {rod_deg}°, trap {trap_deg}°")

    def stop_all_motors(self):
        if self.motors_available:
            self.panel_left.stop()
            self.panel_right.stop()
            self.rod_motor.stop()
            self.trap_motor.stop()
        print("[STOP] All motors stopped.")

# Global instance
garbage_sort_controller = GarbageSortController()
